In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 

def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

In [2]:
data = pd.read_csv("../data//추천장소통합리스트.csv")

In [ ]:
import requests

# 장소 데이터를 예시로 입력 (출발지, 경유지 등)
places = [
    {"name": "출발지", "lat": 37.5665, "lon": 126.9780},
    {"name": "장소1", "lat": 37.5796, "lon": 126.9770},
    # ... 추가 경유지
]

# 거리 및 경로 데이터를 가져오는 함수 (예시용 API 엔드포인트 사용)
def fetch_route_data(origin, destination):
    url = "https://api.example.com/get_route"
    params = {
        "origin": f"{origin['lat']},{origin['lon']}",
        "destination": f"{destination['lat']},{destination['lon']}"
    }
    response = requests.get(url, params=params)
    return response.json()  # {"distance": km, "time": min, "cost": won, "polyline": ...}

# 모든 장소 간 1:1 거리, 시간, 비용 데이터를 수집
routes_data = {}
for i, origin in enumerate(places):
    for j, destination in enumerate(places):
        if i != j:
            routes_data[(i, j)] = fetch_route_data(origin, destination)


In [ ]:
from scipy.spatial.distance import cdist
import numpy as np
from itertools import permutations

# 각 경로에 대한 점수를 계산하는 함수
def calculate_route_score(route, routes_data, weight_distance=0.4, weight_time=0.4, weight_cost=0.2):
    score = 0
    for i in range(len(route) - 1):
        route_info = routes_data[(route[i], route[i + 1])]
        score += (
            weight_distance * route_info["distance"] +
            weight_time * route_info["time"] +
            weight_cost * route_info["cost"]
        )
    return score

# 출발지에서 시작하여 모든 장소를 방문 후 출발지로 돌아오는 최적 경로 탐색
def find_optimal_route(places, routes_data):
    num_places = len(places)
    all_routes = permutations(range(1, num_places))  # 출발지를 제외한 경유지들의 순열 생성
    best_score = float('inf')
    best_route = None
    
    for route in all_routes:
        route = (0,) + route + (0,)  # 출발지(0)를 추가해 순환 경로 형성
        score = calculate_route_score(route, routes_data)
        if score < best_score:
            best_score = score
            best_route = route
            
    return best_route, best_score

# 최적 경로 탐색 수행
best_route, best_score = find_optimal_route(places, routes_data)
print("최적 경로:", best_route)
print("최적 점수:", best_score)


In [ ]:
import folium

# 지도 시각화 예시
m = folium.Map(location=[places[0]["lat"], places[0]["lon"]], zoom_start=13)

# 최적 경로 표시
for i in range(len(best_route) - 1):
    start = places[best_route[i]]
    end = places[best_route[i + 1]]
    polyline = routes_data[(best_route[i], best_route[i + 1])]["polyline"]
    folium.PolyLine(locations=polyline, color="blue", weight=2.5, opacity=1).add_to(m)

# 시작 및 끝 지점 마커 추가
folium.Marker([places[0]["lat"], places[0]["lon"]], popup="출발지").add_to(m)

# 지도 출력
m
